In [3]:

import numpy as np
from scipy.io import loadmat


In [22]:
# def pmhmm(X, T, M=2, K=4, cyc=100, tol=0.0001, iters=10):

X = loadmat('X.mat')['X']
test = loadmat('test.mat')

M=2
K=2
cyc=100
tol=0.0001
iters=10

p = X.shape[1]
N = X.shape[0]
T = N
N = int(N/T)

tiny = test['tiny']
Cov = test['Cov']
XX = test['XX']
Xalt = test['Xalt']

Pi = loadmat('vars.mat')['Pi']
P = loadmat('vars.mat')['P']
Mu = loadmat('vars.mat')['Mu']


k1 = test['k1'] #(dot(2, pi)) ** (- p / 2)
mf = np.ones((N*T,M*K)) / K

alpha = np.zeros((N*T,M*K))
beta = np.zeros((N*T,M*K))
logmf = np.log(mf)
h = np.ones((N*T,M*K)) / K
exph = np.exp(h)



In [23]:
# for cycle in cyc:

cycle = 0
gamma = np.zeros((N*T,M*K))
Eta = np.zeros((M*K,M*K))
Xi = np.zeros((M*K, K))
iCov = np.linalg.inv(Cov)
k2 = k1 / np.sqrt(np.linalg.det(Cov))

itermf = iters

# for l in iters:
l = 0

mfo = np.copy(mf)
logmfo = np.copy(logmf)

# compute h based on mf

for i in range(T):
    # i = 0
    d2 = i
    d = Xalt[d2, :]
    yhat = np.dot(mf[d2, :], Mu)
    for j in range(M):
        d1 = np.arange(j*K, j*K + K)
        Muj = Mu[d1, :]
        h[d2, d1] = Muj @ iCov @ (d-yhat).T + Muj @ iCov @ Muj.T @ mf[d2, d1].T- 0.5 * np.diag(Muj @ iCov @ Muj.T)
        h[d2, d1] = h[d2, d1] - max(h[d2, d1])*np.ones((1,K)) # not quite
exph = np.exp(h)

# compute mf based on h

In [ ]:
scale = np.zeros((T, M))

# for j in range(M):
j = 0
d1 = np.arange(j*K, j*K + K)
d2 = 0
alpha[d2, d1] = exph[d2,d1] * Pi[:,j].T
scale[d2, j] = np.sum(alpha[d2,d1], axis=0) + tiny
alpha[d2, d1] = alpha[d2,d1] / scale[d2,j]
#
for i in range(1,T):
    d2 = i
    alpha[d2,d1] = alpha[d2-1, d1]@P[d1, :]*exph[d2,d1]
    scale[d2, j] = np.sum(alpha[d2,d1], axis=0) + tiny
    alpha[d2, d1] = alpha[d2,d1] / scale[d2,j]


In [64]:

d2=T-1
beta[d2,d1] = np.ones((1, K)) / scale[d2, j]

for i in range(T - 2, -1, -1): # stop at 0
    d2 = i
    beta[d2,d1] = beta[d2 + 1,d1] * exph[d2 + 1,d1] @ P[d1, :].T
    beta[d2,d1] = beta[d2,d1] / scale[d2, j]

mf[:, d1] = alpha[:,d1] * beta[:,d1]
mf[:,d1] = np.divide(mf[:,d1], (np.ones((2, 1)) * np.sum(mf[:,d1], axis=1) + tiny).T)

mf[mf < tiny] = 0
logmf = np.log(mf < tiny)

In [62]:
np.divide(mf[:,d1], (np.ones((2, 1)) * np.sum(mf[:,d1], axis=1) + tiny).T)

array([[0.6125215 , 0.3874785 ],
       [0.69071941, 0.30928059],
       [0.8204097 , 0.1795903 ],
       [0.85959605, 0.14040395],
       [0.82378701, 0.17621299],
       [0.71326252, 0.28673748],
       [0.67677004, 0.32322996],
       [0.68083245, 0.31916755],
       [0.71413847, 0.28586153],
       [0.83608654, 0.16391346],
       [0.81380601, 0.18619399],
       [0.70200336, 0.29799664],
       [0.67142667, 0.32857333],
       [0.66254672, 0.33745328],
       [0.6818652 , 0.3181348 ],
       [0.74332576, 0.25667424],
       [0.84804154, 0.15195846],
       [0.84810395, 0.15189605],
       [0.83970755, 0.16029245],
       [0.71318801, 0.28681199],
       [0.66678204, 0.33321796],
       [0.64821513, 0.35178487],
       [0.6402421 , 0.3597579 ],
       [0.63197909, 0.36802091],
       [0.59614793, 0.40385207],
       [0.4941537 , 0.5058463 ],
       [0.18364771, 0.81635229],
       [0.1191512 , 0.8808488 ],
       [0.09218434, 0.90781566],
       [0.08812382, 0.91187618],
       [0.

In [63]:
(np.ones((2, 1)) * np.sum(mf[:,d1], axis=1) + tiny).T

array([[1.12644196, 1.12644196],
       [1.11038001, 1.11038001],
       [1.35358048, 1.35358048],
       [1.34286527, 1.34286527],
       [1.27446706, 1.27446706],
       [1.09433397, 1.09433397],
       [1.09536669, 1.09536669],
       [1.11086241, 1.11086241],
       [1.10136823, 1.10136823],
       [1.39987775, 1.39987775],
       [1.28048052, 1.28048052],
       [1.08148485, 1.08148485],
       [1.10402374, 1.10402374],
       [1.0945289 , 1.0945289 ],
       [1.10430462, 1.10430462],
       [1.16485979, 1.16485979],
       [1.38120343, 1.38120343],
       [1.27933295, 1.27933295],
       [1.31893831, 1.31893831],
       [1.08862028, 1.08862028],
       [1.09439497, 1.09439497],
       [1.09345233, 1.09345233],
       [1.09521242, 1.09521242],
       [1.10685694, 1.10685694],
       [1.10020808, 1.10020808],
       [1.10428438, 1.10428438],
       [1.65727512, 1.65727512],
       [1.4513006 , 1.4513006 ],
       [1.44768707, 1.44768707],
       [1.43160272, 1.43160272],
       [1.

In [20]:
np.allclose(iCov, test['iCov'])

True